In [1]:
#requirements
import http.client
import json
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import folium
from folium.features import DivIcon
from branca.element import Template, MacroElement
import statistics
import seaborn as sns
from matplotlib import cbook

# Standard library imports
import json
import warnings

# Third-party imports
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from folium import Figure, IFrame, plugins
from folium.plugins import DualMap, HeatMap, MarkerCluster
from shapely import wkt
from shapely.affinity import scale

import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Set up matplotlib to display graphs inline in a Jupyter Notebook
%matplotlib inline

In [24]:
connect_opendata = http.client.HTTPSConnection("opendata-ajuntament.barcelona.cat")

headers = {
    'cache-control': "no-cache"
    }

connect_opendata.request("GET", "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=e42cf2cf-a76e-4a32-9357-cf90e0ea8ead", headers=headers)

response = connect_opendata.getresponse()
data = response.read()

json_data = json.loads(data.decode('utf-8'))

# Check if the JSON data is nested and needs flattening
if 'result' in json_data and 'records' in json_data['result']:
    # Flatten the JSON data and create a DataFrame
    precio_venta_m2 = json_normalize(json_data['result']['records'])
else:
    # Convert the relevant part of your JSON data to a JSON string and read into a DataFrame
    precio_venta_m2 = pd.read_json(json.dumps(json_data['result']['records']))



# Using regular expression to remove the pattern
precio_venta_m2['BARRIS'] = precio_venta_m2['BARRIS'].str.replace(r'^\d+\.\s+', '', regex=True)



precio_venta_m2.columns = ['Nom_Barri', '2011', '2008', '2010','DTE','2009','ID']

precio_venta_m2 = precio_venta_m2.drop(columns=['DTE', 'ID'])

new_order = ['Nom_Barri', '2008', '2009', '2010','2011']
precio_venta_m2 = precio_venta_m2[new_order]

precio_venta_m2.to_csv("precio_venta_m2.csv")

# Display the DataFrame
precio_venta_m2.head()


,Nom_Barri,2008,2009,2010,2011
0,el Raval,3.065,2.773,2.445,2.176
1,el Barri Gòtic,3.828,3.750,3.026,2.632
2,la Barceloneta,--,--,3.005,2.135
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482
4,el Fort Pienc,2.893,2.736,2.422,2.224


In [25]:
import numpy as np

# Convert year columns to numeric
year_columns = precio_venta_m2.columns[1:]  # Exclude the first two and last two columns
precio_venta_m2[year_columns] = precio_venta_m2[year_columns].apply(pd.to_numeric, errors='coerce')

# Calculate the average price per sqm over the years for each neighborhood
precio_venta_m2['Average_Price'] = precio_venta_m2[year_columns].mean(axis=1)

precio_venta_m2.to_csv("precio_venta_m2.csv")

# Display the updated dataframe
precio_venta_m2.head()


,Nom_Barri,2008,2009,2010,2011,Average_Price
0,el Raval,3.065,2.773,2.445,2.176,2.61475
1,el Barri Gòtic,3.828,3.750,3.026,2.632,3.30900
2,la Barceloneta,NaN,NaN,3.005,2.135,2.57000
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482,2.97650
4,el Fort Pienc,2.893,2.736,2.422,2.224,2.56875


In [26]:
# Descriptive statistics for the dataset
descriptive_stats = precio_venta_m2.describe()

# Display descriptive statistics
descriptive_stats

,2008,2009,2010,2011,Average_Price
count,56.000000,53.000000,56.00000,58.000000,61.000000
mean,2.705982,2.440208,2.28775,2.004810,2.330217
std,0.685203,0.546705,0.55166,0.464295,0.538360
min,1.491000,1.387000,1.28900,1.110000,1.432250
25%,2.174250,2.058000,1.90950,1.681750,1.900250
50%,2.670500,2.392000,2.28250,1.974000,2.251500
75%,3.142750,2.773000,2.66175,2.264000,2.672750
max,4.326000,3.750000,3.98900,3.351000,3.848250


The year 2009 has the most missing values (20 instances).
The year 2011 has the fewest missing values (15 instances).
Missing data in other years ranges from 15 to 20 instances.
Now, I'll provide a brief overview of this dataset, including descriptive statistics and any other interesting observations. Let's start with the descriptive statistics. ​​

The descriptive statistics provide insight into the price data for commercial properties in Barcelona from 2008 to 2011. Here are some key observations:

Count: The dataset contains 73 neighborhoods. However, not all years have data for every neighborhood, as indicated by the varying counts (56 for 2008, 53 for 2009, 56 for 2010, and 58 for 2011).

Mean Prices:

The average price per square meter was highest in 2008 (≈2.706 ≈2.706 EUR), and it decreased each subsequent year, reaching its lowest in 2011 (≈2.005≈2.005 EUR).

The overall average price across all years and neighborhoods is approximately 2.330 EUR.
Standard Deviation:

The standard deviation indicates variability in prices across neighborhoods. The highest variability was in 2008 (≈0.685≈0.685) and the lowest in 2011 (≈0.464≈0.464).

Minimum and Maximum Prices:

The minimum price per square meter fluctuated from 1.491 EUR in 2008 to 1.110 EUR in 2011.
The maximum price showed a decreasing trend, from 4.326 EUR in 2008 to 3.351 EUR in 2011.
Quartiles:

The 25th, 50th (median), and 75th percentiles also exhibit a decreasing trend across the years.
This analysis reveals a general decrease in the price per square meter for commercial properties in Barcelona's neighborhoods from 2008 to 2011. The variability in prices also decreased over this period, suggesting a possible stabilization or uniformity in property values across different areas.




In [27]:
precio_venta_m2.drop(['Average_Price'], axis=1)


,Nom_Barri,2008,2009,2010,2011
0,el Raval,3.065,2.773,2.445,2.176
1,el Barri Gòtic,3.828,3.750,3.026,2.632
2,la Barceloneta,NaN,NaN,3.005,2.135
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482
4,el Fort Pienc,2.893,2.736,2.422,2.224
...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,3.752,2.445,2.991,2.651
69,el Besòs i el Maresme,3.669,2.842,2.751,2.121
70,Provençals del Poblenou,3.857,2.660,2.516,2.255
71,Sant Martí de Provençals,1.957,2.266,2.175,1.812


In [28]:
precio_venta_m2=pd.melt(precio_venta_m2, id_vars=['Nom_Barri'], value_vars=['2008', '2009','2010','2011'])

precio_venta_m2.columns = ['Nom_Barri', 'Anio', 'Price']

precio_venta_m2

,Nom_Barri,Anio,Price
0,el Raval,2008,3.065
1,el Barri Gòtic,2008,3.828
2,la Barceloneta,2008,NaN
3,Sant Pere Santa Caterina i la Ribera,2008,3.450
4,el Fort Pienc,2008,2.893
...,...,...,...
287,Diagonal Mar i el Front Marítim del Poblenou,2011,2.651
288,el Besòs i el Maresme,2011,2.121
289,Provençals del Poblenou,2011,2.255
290,Sant Martí de Provençals,2011,1.812


## Precio Alquiler

In [30]:
connect_opendata = http.client.HTTPSConnection("opendata-ajuntament.barcelona.cat")

headers = {
    'cache-control': "no-cache"
    }

connect_opendata.request("GET", "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=97356d26-30b9-436a-8dbb-d0d05f0a87fd", headers=headers)

response = connect_opendata.getresponse()
data = response.read()

json_data = json.loads(data.decode('utf-8'))

# Check if the JSON data is nested and needs flattening
if 'result' in json_data and 'records' in json_data['result']:
    # Flatten the JSON data and create a DataFrame
    precio_alquiler = json_normalize(json_data['result']['records'])
else:
    # Convert the relevant part of your JSON data to a JSON string and read into a DataFrame
    precio_alquiler = pd.read_json(json.dumps(json_data['result']['records']))



# Using regular expression to remove the pattern
precio_alquiler['BARRIS'] = precio_venta_m2['BARRIS'].str.replace(r'^\d+\.\s+', '', regex=True)



precio_alquiler.columns = ['Nom_Barri', '2011', '2008', '2010','DTE','2009','ID']

precio_alquiler = precio_alquiler.drop(columns=['DTE', 'ID'])

new_order = ['Nom_Barri', '2008', '2009', '2010','2011']
precio_alquiler = precio_alquiler[new_order]

precio_alquiler.to_csv("precio_alquiler.csv")

# Display the DataFrame
precio_alquiler.head()

,Nom_Barri,2008,2009,2010,2011
0,el Raval,1543,1362,1216,1189
1,el Barri Gòtic,1859,1481,1369,1325
2,la Barceloneta,1469,1526,1660,1648
3,Sant Pere Santa Caterina i la Ribera,1585,1374,1291,1225
4,el Fort Pienc,1243,1098,1000,921


In [32]:
precio_alquiler=pd.melt(precio_alquiler, id_vars=['Nom_Barri'], value_vars=['2008', '2009','2010','2011'])

precio_alquiler.columns = ['Nom_Barri', 'Anio', 'Price']

precio_alquiler

,Nom_Barri,Anio,Price
0,el Raval,2008,1543
1,el Barri Gòtic,2008,1859
2,la Barceloneta,2008,1469
3,Sant Pere Santa Caterina i la Ribera,2008,1585
4,el Fort Pienc,2008,1243
...,...,...,...
287,Diagonal Mar i el Front Marítim del Poblenou,2011,1126
288,el Besòs i el Maresme,2011,--
289,Provençals del Poblenou,2011,832
290,Sant Martí de Provençals,2011,976
